## LeNet

In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from tensorflow.keras.regularizers import l2
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adadelta

In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11490434/11490434 [==============================] - 2s 0us/step


In [3]:
img_rows = x_train[0].shape[0]
img_cols = x_train[1].shape[0]

In [5]:
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)

In [6]:
input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype("float32")
x_test = x_test.astype("float32")

x_train /= 255
x_test /= 255

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

num_classes = y_test.shape[1]
num_pixels = x_train.shape[1]*x_train.shape[2]

In [10]:
model = Sequential()

model.add(Conv2D(6, (5, 5),
                 padding="same",
                 input_shape=input_shape))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Conv2D(16, (5, 5),
                 padding="same"))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Conv2D(120, (5, 5),
                 padding="same"))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

model.add(Flatten())
model.add(Dense(120))
model.add(Activation("relu"))

model.add(Dense(84))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation("softmax"))

model.compile(loss="categorical_crossentropy",
               optimizer=Adadelta(),
               metrics=['accuracy'])

print(model.summary())

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (None, 28, 28, 6)         156       
                                                                 
 activation_18 (Activation)  (None, 28, 28, 6)         0         
                                                                 
 max_pooling2d_9 (MaxPoolin  (None, 14, 14, 6)         0         
 g2D)                                                            
                                                                 
 conv2d_10 (Conv2D)          (None, 14, 14, 16)        2416      
                                                                 
 activation_19 (Activation)  (None, 14, 14, 16)        0         
                                                                 
 max_pooling2d_10 (MaxPooli  (None, 7, 7, 16)          0         
 ng2D)                                                

In [11]:
batch_size = 128
epochs = 50

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    validation_data=(x_test, y_test),
                    shuffle=True)

scores = model.evaluate(x_test, y_test, verbose=1)
print("Test loss: ", scores[0])
print("Test accuracy: ", scores[1])

Epoch 1/50
469/469 [==============================] - 8s 8ms/step - loss: 2.2988 - accuracy: 0.1281 - val_loss: 2.2950 - val_accuracy: 0.1508
Epoch 2/50
469/469 [==============================] - 3s 6ms/step - loss: 2.2903 - accuracy: 0.1982 - val_loss: 2.2865 - val_accuracy: 0.2308
Epoch 3/50
469/469 [==============================] - 2s 5ms/step - loss: 2.2825 - accuracy: 0.2631 - val_loss: 2.2786 - val_accuracy: 0.2749
Epoch 4/50
469/469 [==============================] - 2s 5ms/step - loss: 2.2744 - accuracy: 0.2964 - val_loss: 2.2695 - val_accuracy: 0.3084
Epoch 5/50
469/469 [==============================] - 2s 5ms/step - loss: 2.2646 - accuracy: 0.3358 - val_loss: 2.2584 - val_accuracy: 0.3570
Epoch 6/50
469/469 [==============================] - 3s 6ms/step - loss: 2.2525 - accuracy: 0.3905 - val_loss: 2.2446 - val_accuracy: 0.4242
Epoch 7/50
469/469 [==============================] - 3s 6ms/step - loss: 2.2374 - accuracy: 0.4594 - val_loss: 2.2275 - val_accuracy: 0.4796
Epoch 

## AlexNet

In [12]:
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adadelta
from tensorflow.keras.utils import to_categorical

In [13]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

num_classes = 10
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

170498071/170498071 [==============================] - 13s 0us/step
x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [15]:
l2_reg = 0.001

model = Sequential()

# 1st layer
model.add(Conv2D(96, (11, 11), input_shape=x_train.shape[1:],
                 padding="same", kernel_regularizer=l2(l2_reg)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# 2nd layer
model.add(Conv2D(256, (5, 5), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# 3rd layer
model.add(ZeroPadding2D((1, 1)))
model.add(Conv2D(512, (3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# 4th layer
model.add(ZeroPadding2D((1, 1)))
model.add(Conv2D(1024, (3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

# 5th layer
model.add(ZeroPadding2D((1, 1)))
model.add(Conv2D(1024, (3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# 1st FC layer
model.add(Flatten())
model.add(Dense(3072))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))

# 2nd FC layer
model.add(Dense(4096))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))

# 3rd FC layer
model.add(Dense(num_classes))
model.add(BatchNormalization())
model.add(Activation('softmax'))

print(model.summary())

model.compile(loss='categorical_crossentropy',
              optimizer=Adadelta(),
              metrics=['accuracy'])

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_17 (Conv2D)          (None, 32, 32, 96)        34944     
                                                                 
 batch_normalization_8 (Bat  (None, 32, 32, 96)        384       
 chNormalization)                                                
                                                                 
 activation_32 (Activation)  (None, 32, 32, 96)        0         
                                                                 
 max_pooling2d_16 (MaxPooli  (None, 16, 16, 96)        0         
 ng2D)                                                           
                                                                 
 conv2d_18 (Conv2D)          (None, 16, 16, 256)       614656    
                                                                 
 batch_normalization_9 (Bat  (None, 16, 16, 256)      

In [16]:
batch_size = 64
epochs = 25

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    validation_data=(x_test, y_test),
                    shuffle=True)

scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss: ', scores[0])
print('Test accuracy: ', scores[1])

Epoch 1/25
782/782 [==============================] - 104s 117ms/step - loss: 2.1449 - accuracy: 0.2453 - val_loss: 1.7713 - val_accuracy: 0.3866
Epoch 2/25
782/782 [==============================] - 87s 112ms/step - loss: 1.8507 - accuracy: 0.3415 - val_loss: 1.6727 - val_accuracy: 0.4296
Epoch 3/25
782/782 [==============================] - 87s 111ms/step - loss: 1.7426 - accuracy: 0.3844 - val_loss: 1.6089 - val_accuracy: 0.4569
Epoch 4/25
782/782 [==============================] - 87s 112ms/step - loss: 1.6831 - accuracy: 0.4138 - val_loss: 1.5621 - val_accuracy: 0.4753
Epoch 5/25
782/782 [==============================] - 88s 112ms/step - loss: 1.6333 - accuracy: 0.4352 - val_loss: 1.5239 - val_accuracy: 0.4930
Epoch 6/25
782/782 [==============================] - 87s 112ms/step - loss: 1.5945 - accuracy: 0.4548 - val_loss: 1.4935 - val_accuracy: 0.5028
Epoch 7/25
782/782 [==============================] - 88s 112ms/step - loss: 1.5576 - accuracy: 0.4693 - val_loss: 1.4693 - val_a